In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
ls checkpoints

batok_13200_safetesnors/  Meta-Llama-3.1-8B-Instruct/


In [10]:
path = "checkpoints"
tokenizer_o = AutoTokenizer.from_pretrained(f"{path}/Meta-Llama-3.1-8B-Instruct")
tokenizer_b = AutoTokenizer.from_pretrained(f"{path}/batok_13200_safetesnors")

In [11]:
tokenizer_o.chat_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

In [12]:
tokenizer_b.chat_template

In [1]:
from prompt.prompt import create_prompt

In [ ]:
# llm-recipes/finetuning.py --model_name checkpoints/Meta-Llama-3.1-8B-Instruct/ --dataset.file datasets/loader/squad.py --lr 1e-6 --num_epochs 5 --batch_size_training 4 --val_batch_size 4 --output_dir train/output/path --distillation_config.model_name checkpoints/batok_13200_safetesnors --distillation --distillation_config.enable_fsdp --distillation_config.pure_bf16 --distillation_config.distil_factor 1.5 --save_step 100

In [ ]:
# CUDA_VISIBLE_DEVICES=2,3 torchrun --nproc-per-node=2 finetuning.py --model_name checkpoints/Meta-Llama-3.1-8B-Instruct/ --dataset.file datasets/loader/squad.py --lr 1e-6 --num_epochs 5 --batch_size_training 4 --val_batch_size 4 --output_dir train/output/path --distillation_config.model_name checkpoints/batok_13200_safetesnors --distillation --distillation_config.enable_fsdp --distillation_config.pure_bf16 --distillation_config.distil_factor 1.5 --save_step 100au

In [1]:
from datasets import load_dataset

dataset = load_dataset('Nicolas-BZRD/uld_loss_Llama-2-7b-chat-hf-squad', data_dir='data')

In [3]:
df = dataset['validation'].to_pandas()

In [4]:
df

,context,question,answers,answers_generated
0,"Dell's manufacturing process covers assembly, ...",Where did Dell configure their notebooks?,"{'answer_start': [287], 'text': ['in-house']}",In-house.
1,A second problem with materialism is that it o...,What is another issue that Whitehead had with ...,"{'answer_start': [42], 'text': ['it obscures t...",Relationships
2,Major religious groups in the Republic of the ...,What percentage of Marshall Islanders worship ...,"{'answer_start': [367], 'text': ['2.2%']}",2.2%
3,"Following the 1914–15 edition, the competition...",what year did competition resume after world w...,"{'answer_start': [113], 'text': ['1919–20.']}",1919
4,Frustration with censorship and editorial inte...,Pilote cartoonists were upset with censorship ...,"{'answer_start': [32], 'text': ['editorial int...",editorial interference
...,...,...,...,...
4375,Although used in the Census and the American C...,"In addition to the Census, where else is the ""...","{'answer_start': [32], 'text': ['the American ...","Other places where ""some other race"" is an opt..."
4376,After a nonprofit organization has been formed...,Does this tax exempt status apply to other taxes?,"{'answer_start': [804], 'text': ['must pay fed...",No taxes.
4377,Details of school casualties had been under no...,How many students were disabled in Xinhua?,"{'answer_start': [450], 'text': ['546']}",546
4378,"In 1903, Josiah White III bought a parcel of l...",In what year did Josiah White III by land to b...,"{'answer_start': [3], 'text': ['1903']}",1903


In [3]:
from datasets import load_dataset, Dataset, DatasetDict

ds = load_dataset('json', data_files="/data/nvme7n1p1/critical_all/critical_instruct_1/2024.09.23_instruct_kk_QnA-hotpot-qa-train.json_31300907.json")

In [6]:
ds_split = ds['train'].train_test_split(test_size=10) 
ds_to_upload = DatasetDict({"train":ds_split['test'], "validation":ds_split['test']})

In [8]:
from huggingface_hub import HfApi, login

api = HfApi()
repo_id = "AdalAbilbekov/test_v1"
api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)

ds_to_upload.push_to_hub(repo_id, private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00, 13.84it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/AdalAbilbekov/test_v1/commit/0441e488bcf6fcbbc00583ed06eac1bcb4cb95af', commit_message='Upload dataset', commit_description='', oid='0441e488bcf6fcbbc00583ed06eac1bcb4cb95af', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from transformers import AutoModelForCausalLM

model_a=AutoModelForCausalLM.from_pretrained('/data/nvme3n1p1/adal_workspace/distilation_process/llm-recipes/checkpoints/Meta-Llama-3.1-8B-Instruct')
model_b=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
# model_c=AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m-deduped")
model_c_28 = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-2.8b-deduped")
model_2_inst = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


In [14]:
model_a

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb

In [15]:
model_c_28

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=Tr

In [18]:
model_2_inst

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb): Ll

In [21]:
from transformers import AutoTokenizer

tokenizer3_2 = AutoTokenizer.from_pretrained("checkpoints/Meta-Llama-3.1-8B-Instruct")
tokenizer3_1 = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

tokenizers = [tokenizer3_2, tokenizer3_1]

In [25]:
tokenizer3_2.get_vocab() == tokenizer3_1.get_vocab()

True

In [22]:
text = "Қазақстан"

for tokenizer in tokenizers:
    print(tokenizer.encode(text))

[128000, 142, 248, 109424, 142, 249, 52037]
[128000, 142, 248, 109424, 142, 249, 52037]


{'Hello': 'Mark'}

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('AdalAbilbekov/test_v1', data_dir='data')

In [7]:
dataset['train'][0]

{'instruction': 'Мезоамерикандық хронология Колумбияға дейінгі Месоамерика тарихын бірнеше кезеңдерге бөледі: палео-үнді (адамның алғашқы қоныстануы – б.з.б. 3500), архаикалық (б.з.б. 3500–2000), классикалыққа дейінгі немесе қалыптасу (б.з.б. 2000–200), классикалық. (200 ж. – 1000 ж.), және Постклассик (б. з. 1000 – 1697 ж.). Кейбір кезеңдік бөлімдер Майя тарихынан алынған: Классикалық-Классикалық шекара бірінші Майяның «құлдырауын», Классикалық-Постклассикалық шекара екіншісін, ал 1697 жылдың аяқталу күні соңғы тәуелсіз Майяның жаулап алуын білдіреді. қала-мемлекет, Таясал. Дегенмен, бұл хронология Колумбияға дейінгі басқа мезоамерикандық өркениеттерге де қатысты. Педро Армиллас Гарсиа (9 қыркүйек 1914 жыл – 11 сәуір 1984 жыл) — испандық академиялық антрополог, археолог және 20 ғасырдың ортасындағы Колумбияға дейінгі ықпалды месоамерикандық ғалым. Археолог ретінде ол Мексиканың орталық және солтүстігіндегі көптеген жерлерде дала жұмыстары мен қазба жұмыстарымен және археологиялық теор

In [9]:
dataset['train'].to_json('/data/nvme3n1p1/adal_workspace/checkv1/v1.json', lines=True,force_ascii=False)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 71.52ba/s]


87197

In [11]:
from transformers import AutoModel

model = AutoModel.from_pretrained("/data/nvme3n1p1/adal_workspace/distilation_process/llm-recipes/checkpoints/distilled_model/1500")

In [13]:
ds = load_dataset("AdalAbilbekov/llama_chat_issai", data_dir="data")

In [16]:
df = ds['train'].to_pandas()

In [18]:
df[df['answers_generated'].str.contains('Com')]

,context,question,answers_generated
217,"One Commerce Plaza, сонымен қатар егіз мұнарал...",Альфред Э. Смит ғимараты Нью-Йорк штатының Кап...,One Commerce Plaza
267,Американдық Уолтхэм Сағат компаниясының тарихи...,Уолтхэм Массачусетс штатындағы 110 Crescent St...,Waltham Watch Company
432,Халика - унитарлық универсалистер тойлайтын ме...,"«Алты күн, жеті түн» фильмін қандай продюсерлі...",Montecito Picture Company
483,Отлиле «Оти» Мабусе (1990 жылы 8 тамызда туған...,1980 жылы 19 қаңтарда дүниеге келген ресейлік ...,Strictly Come Dancing
688,American Business Journal – ағымдағы бизнес та...,1912 жылға дейін ең табысты американдық және х...,EP Charlton & Company
1456,Elektor теледидар ойындарының компьютері (TVGC...,Elektor TV Games компьютері Sharp корпорациясы...,Elektor TV Games Computer (TVGC) 1979 жылдан ...
1503,Las Vegas Premium Outlets North - бұл 540 000 ...,Jersey Gardens-тегі The Mills немесе Woodbury ...,Woodbury Common Premium сауда нүктелері
2233,HOY Regional — Хуануко қаласында шығатын Перу ...,"Ace of Spades штаб-пәтеріне сілтеме жасап, сіл...",Dow Jones & Company
3379,2013 жылғы Англия кубогының финалы Англия кубо...,Бұл 2003 жылғы Қытай футбол кубогының титулдық...,"CBR Brewing Company, Inc."
4274,Robot Chicken DC Comics Special — «Robot Chick...,2012 жылы ересектер үшін жүзуде көрсетілген жи...,Robot Chicken DC Comics Арнайы


In [ ]:
# Com', ' Tru', 'ise